# Spin tests

This notebook contains the code used to perform the spin-tests contained in the study. Note that it **not** necessary to run this notebook to obtain the spin-test data, since it was already precomputed and stored in the folder `spin-test`.

In [ ]:
import nibabel as nib
import pandas as pd
import numpy as np
import os
import scipy.io
from scipy.stats import pearsonr, spearmanr

In [ ]:
def compute_rot_metrics_on_annot(map_data_lh, map_data_rh,
                                 annot_lh, annot_rh,
                                 rot_indices_lh, rot_indices_rh,
                                 agg_function, annot_parcel_names_dict=None,
                                 join_hemi=True):
    """Computes the metrics provided in the maps aggregated on the parcellations provided in the annotation files for each spin permutation"""

    df_lh = pd.DataFrame({'map_value': map_data_lh, 'parcel': annot_lh[0].astype('int64')})
    df_rh = pd.DataFrame({'map_value': map_data_rh, 'parcel': annot_rh[0].astype('int64')})

    if annot_parcel_names_dict is not None:
        df_lh['parcel_name'] = df_lh.apply(lambda x: annot_parcel_names_dict[annot_lh[2][int(x.parcel)].decode()], axis=1)
        df_rh['parcel_name'] = df_rh.apply(lambda x: annot_parcel_names_dict[annot_rh[2][int(x.parcel)].decode()], axis=1)
    else:
        df_lh['parcel_name'] = df_lh.apply(lambda x: annot_lh[2][int(x.parcel)].decode(), axis=1)
        df_rh['parcel_name'] = df_rh.apply(lambda x: annot_rh[2][int(x.parcel)].decode(), axis=1)

    if not join_hemi:
        df_lh['parcel_name'] = "lh_" + df_lh['parcel_name']
        df_rh['parcel_name'] = "rh_" + df_rh['parcel_name']
        
    df_bh = pd.concat((df_lh, df_rh))
    df_bh.drop(columns='parcel', inplace=True)

    df_aux = df_bh.groupby('parcel_name').mean().reset_index()
    df_aux['rotation'] = 'original'

    df_rot_metrics = df_aux.copy()

    for i, (p_lh, p_rh) in enumerate(zip(rot_indices_lh, rot_indices_rh)):
        df_aux = pd.DataFrame({f'map_value': np.concatenate((map_data_lh[p_lh - 1], map_data_rh[p_rh - 1]))})
        df_aux = pd.concat((df_bh.reset_index(drop=True)['parcel_name'], df_aux), axis=1).groupby('parcel_name').agg(agg_function).reset_index()
        df_aux['rotation'] = f'rot{i}'
        df_rot_metrics = pd.concat((df_rot_metrics, df_aux.copy()))
    return df_rot_metrics


def compute_rot_metrics_on_maps(map_data_lh, map_data_rh,
                                map_target_data_lh, map_target_data_rh,
                                rot_indices_lh, rot_indices_rh,
                                corr_function=pearsonr):
    """Computes the correlation of a map with respect to a targer map for each spin permutation"""

    map_data = np.concatenate((map_data_lh, map_data_rh))
    map_target_data = np.concatenate((map_target_data_lh, map_target_data_rh))
    
    nas = np.logical_or(np.isnan(map_data), np.isnan(map_target_data))
    corr, _ = corr_function(map_data[~nas], map_target_data[~nas])
    df_aux = pd.DataFrame({'rotation': ['original'], 'corr_metric': [corr]})
    df_rot_metrics = df_aux.copy()

    for i, (p_lh, p_rh) in enumerate(zip(rot_indices_lh, rot_indices_rh)):
        map_data_rot = np.concatenate((map_data_lh[p_lh - 1], map_data_rh[p_rh - 1]))
        nas = np.logical_or(np.isnan(map_data_rot), np.isnan(map_target_data))
        df_aux['rotation'] = f'rot{i}'
        df_aux['corr_metric'], _ = corr_function(map_data_rot[~nas], map_target_data[~nas])
        df_rot_metrics = pd.concat((df_rot_metrics, df_aux.copy()))
    return df_rot_metrics


def compute_spin_test_p_values(df_rot_metrics, column_name='map_value', tail='positive'):
    """Computes the p-values of the spin-test"""
    cols_groupby = df_rot_metrics.columns.difference({'rotation', column_name}).to_list()
    if tail=='positive':
        df_spin_test_p_values = df_rot_metrics.groupby(cols_groupby).apply(lambda x: np.mean(x.query('rotation=="original"')[column_name].values <= x[column_name].values))
    elif tail=='negative':
        df_spin_test_p_values = df_rot_metrics.groupby(cols_groupby).apply(lambda x: np.mean(x.query('rotation=="original"')[column_name].values >= x[column_name].values))   
    df_spin_test_p_values = df_spin_test_p_values.reset_index(name='p_value')
    df_spin_test_p_values['tail'] = tail
    return df_spin_test_p_values

# Spin-tests for significance maps and effect size maps using the seven resting-state networks described in Yeo et al. 2011

In [ ]:
data_dir = os.getcwd()
annot_name = 'Yeo2011_7Networks_N1000'
annot_lh = nib.freesurfer.read_annot(f"{data_dir}/fsaverage/label/lh.{annot_name}.annot")
annot_rh = nib.freesurfer.read_annot(f"{data_dir}/fsaverage/label/rh.{annot_name}.annot")

# Spin permutations of fsaverage index computed with https://github.com/spin-test/spin-test
# This large file (~1Gb) is not included in the repository, it can be found in:
# https://www.dropbox.com/scl/fi/fzqz430auo3535p6mn35m/fsaverage_indices_1000_spin_permutations.mat?rlkey=y6o28kd8ojkj4a42ntva3o3ma&dl=0
rot_indices_file = f"{data_dir}/spin-test/fsaverage_indices_1000_spin_permutations.mat"
mat = scipy.io.loadmat(rot_indices_file)
rot_indices_lh = mat['indexrotl']
rot_indices_rh = mat['indexrotr']
del mat

# These are name replacements for the Yeo 7 Networks
net_replacements = {'FreeSurfer_Defined_Medial_Wall': 'medial_wall',
                    '7Networks_1': 'visual', 
                    '7Networks_2': 'somatomotor',
                    '7Networks_3': 'dorsal_attention',
                    '7Networks_4': 'ventral_attention',
                    '7Networks_5': 'limbic',
                    '7Networks_6': 'fronto_parietal',
                    '7Networks_7': 'default_mode'}

data_sets = ['main_data', 'replication_data']
models = {'main_data': ['without_covariates', 'covariates_euler_age_etiv', 'covariates_euler_age_etiv_psqi_pss'],
          'replication_data': ['without_covariates', 'covariates_euler_age_etiv']}
contrasts = ['sesPrg', 'sesPost', 'groupxses']
metrics = ['volume', 'thickness', 'area']
map_metrics = ['partialETA2', 'signed_partialETA2', 'thr_FDRcorrP_across_hemi']
agg_metric = 'mean'
map_file_pattern = "{}/{}/{}/{}.{}_sm10_{}_contrast_{}.mgh"
mask_file_pattern = "{}/{}/{}.mask.mgh"


dfs = []
dfs_pvalues = []
for ds in data_sets:
    for mod in models[ds]:
        for c in contrasts:
            for m in metrics:
                for mm in map_metrics:
                    mm_aux = mm.replace("thr_", "").replace("signed_", "")
                    print(ds ,mod, c, m, mm)
                    map_data_lh = nib.load(map_file_pattern.format(data_dir, ds, mod, "lh", m, mm_aux, c)).get_fdata().ravel()
                    map_data_rh = nib.load(map_file_pattern.format(data_dir, ds, mod, "rh", m, mm_aux, c)).get_fdata().ravel()
                    mask_lh = nib.load(mask_file_pattern.format(data_dir, ds, 'lh')).get_fdata().astype('bool').ravel()
                    mask_rh = nib.load(mask_file_pattern.format(data_dir, ds, 'rh')).get_fdata().astype('bool').ravel()      
                    map_data_lh[np.logical_not(mask_lh)]=np.nan  # Masking vertices outside the mask
                    map_data_rh[np.logical_not(mask_rh)]=np.nan  # Masking vertices outside the mask
                    if "signed_" in mm:
                        sgn_data_lh = nib.load(map_file_pattern.format(data_dir, ds, mod, "lh", m, 'sgn', c)).get_fdata().ravel()
                        sgn_data_rh = nib.load(map_file_pattern.format(data_dir, ds, mod, "rh", m, 'sgn', c)).get_fdata().ravel()
                        map_data_lh = sgn_data_lh * map_data_lh
                        map_data_rh = sgn_data_rh * map_data_rh
                    if "thr_" in mm:
                        map_data_lh = (map_data_lh > 0.95)*100  # Thresholding the significance map at alpha = 0.05 and scaling to obtain percentage
                        map_data_rh = (map_data_rh > 0.95)*100  # Thresholding the significance map at alpha = 0.05 and scaling to obtain percentage
                    df_rot_metrics = compute_rot_metrics_on_annot(map_data_lh, map_data_rh,
                                                                  annot_lh, annot_rh,
                                                                  rot_indices_lh, rot_indices_rh,
                                                                  agg_metric, net_replacements, join_hemi=True)
                    df_rot_metrics['data_set'] = ds
                    df_rot_metrics['model'] = mod
                    df_rot_metrics['contrast'] = c
                    df_rot_metrics['metric'] = m
                    df_rot_metrics['map_metric'] = mm
                    df_rot_metrics['agg_metric'] = agg_metric
                    df_rot_metrics['num_rots'] = rot_indices_lh.shape[0]
                    dfs.append(df_rot_metrics)
                    dfs_pvalues.append(compute_spin_test_p_values(df_rot_metrics, tail='positive'))
                    dfs_pvalues.append(compute_spin_test_p_values(df_rot_metrics, tail='negative'))
df_rot_metrics = pd.concat(dfs)
df_pvalues = pd.concat(dfs_pvalues)
del dfs, dfs_pvalues

In [ ]:
column_order_rot_metrics = ['data_set', 'model', 'metric', 'contrast', 'map_metric', 'agg_metric', 'num_rots', 'rotation', 'parcel_name', 'map_value']
column_order_pvalues = ['data_set', 'model', 'metric', 'contrast', 'map_metric', 'agg_metric', 'num_rots', 'parcel_name', 'tail', 'p_value']
df_rot_metrics.reset_index(drop=True).loc[:,column_order_rot_metrics].to_feather(f"{data_dir}/spin-test/yeo_spin_test_data.feather")
df_pvalues.reset_index(drop=True).loc[:,column_order_pvalues].to_feather(f"{data_dir}/spin-test/yeo_spin_test_pvalues.feather")

# Spin-tests for dataset results comparison

In [ ]:
data_dir = os.getcwd()
main_data_dir = f"{data_dir}/main_data"
replication_data_dir = f"{data_dir}/replication_data"
models = ['without_covariates', 'covariates_euler_age_etiv']
contrasts = ['sesPrg', 'sesPost', 'groupxses']
metrics = ['volume', 'thickness', 'area']
map_metrics = [ 'partialETA2', 'signed_partialETA2']
corr_functions = [pearsonr, spearmanr]
mask_main_lh = nib.load("{}/lh.mask.mgh".format(main_data_dir)).get_fdata().astype('bool').ravel()
mask_main_rh = nib.load("{}/rh.mask.mgh".format(main_data_dir)).get_fdata().astype('bool').ravel()
mask_replication_lh = nib.load("{}/lh.mask.mgh".format(replication_data_dir)).get_fdata().astype('bool').ravel()
mask_replication_rh = nib.load("{}/rh.mask.mgh".format(replication_data_dir)).get_fdata().astype('bool').ravel()


# Spin permutations of fsaverage index computed with https://github.com/spin-test/spin-test
# This large file (~1Gb) is the same as for the above spin-tests and it is not included in the repository, it can be found in:
# https://www.dropbox.com/scl/fi/fzqz430auo3535p6mn35m/fsaverage_indices_1000_spin_permutations.mat?rlkey=y6o28kd8ojkj4a42ntva3o3ma&dl=0
rot_indices_file = "./spin-test/fsaverage_indices_1000_spin_permutations.mat"
mat = scipy.io.loadmat(rot_indices_file)
rot_indices_lh = mat['indexrotl']
rot_indices_rh = mat['indexrotr']
del mat

str_image_format = "{}/{}/{}.{}_sm10_{}_contrast_{}.mgh"
             
dfs = []
dfs_corr_pvalue = []
for mod in models:
    for c in contrasts:
        for m in metrics:
            for mm in map_metrics:
                mm_aux = mm.replace('signed_', '')
                for cf in corr_functions:
                    print(mod, c, m, mm, cf.__name__)
                    main_map_lh = nib.load(str_image_format.format(main_data_dir, mod, "lh", m, mm_aux, c)).get_fdata().ravel()
                    main_map_rh = nib.load(str_image_format.format(main_data_dir, mod, "rh", m, mm_aux, c)).get_fdata().ravel()
                    replication_map_lh = nib.load(str_image_format.format(replication_data_dir, mod, "lh", m, mm_aux, c)).get_fdata().ravel()
                    replication_map_rh = nib.load(str_image_format.format(replication_data_dir, mod, "rh", m, mm_aux, c)).get_fdata().ravel()
                    if 'signed_' in mm:
                        main_map_sgn_lh = nib.load(str_image_format.format(main_data_dir, mod, "lh", m, 'sgn', c)).get_fdata().ravel()
                        main_map_sgn_rh = nib.load(str_image_format.format(main_data_dir, mod, "rh", m, 'sgn', c)).get_fdata().ravel()
                        main_map_lh = main_map_lh*main_map_sgn_lh
                        main_map_rh = main_map_rh*main_map_sgn_rh
                        replication_map_sgn_lh = nib.load(str_image_format.format(replication_data_dir, mod, "lh", m, 'sgn', c)).get_fdata().ravel()
                        replication_map_sgn_rh = nib.load(str_image_format.format(replication_data_dir, mod, "rh", m, 'sgn', c)).get_fdata().ravel()
                        replication_map_lh = replication_map_lh*replication_map_sgn_lh
                        replication_map_rh = replication_map_rh*replication_map_sgn_rh
                    main_map_lh[np.logical_not(mask_main_lh)] = np.nan  # Masking vertices outside the mask
                    main_map_rh[np.logical_not(mask_main_rh)] = np.nan  # Masking vertices outside the mask
                    replication_map_lh[np.logical_not(mask_replication_lh)] = np.nan  # Masking vertices outside the mask
                    replication_map_rh[np.logical_not(mask_replication_rh)] = np.nan  # Masking vertices outside the mask
                    df_corr = compute_rot_metrics_on_maps(main_map_lh, main_map_rh, 
                                                          replication_map_lh, replication_map_rh,
                                                          rot_indices_lh, rot_indices_rh, cf)
                    df_corr['model'] = mod
                    df_corr['contrast'] = c
                    df_corr['metric'] = m
                    df_corr['map_metric'] = mm
                    df_corr['num_rots'] = rot_indices_lh.shape[0]
                    df_corr['corr_function'] = cf.__name__
                    dfs.append(df_corr)
                    dfs_corr_pvalue.append(compute_spin_test_p_values(df_corr, 'corr_metric', tail='positive'))
    df_corr = pd.concat(dfs)
    df_corr_pvalues = pd.concat(dfs_corr_pvalue)
del dfs, dfs_corr_pvalue

In [ ]:
column_order_corr = ['model', 'metric', 'contrast', 'map_metric', 'corr_function', 'num_rots', 'rotation', 'corr_metric']
column_order_corr_pvalues = ['model', 'metric', 'contrast', 'map_metric', 'corr_function', 'num_rots', 'tail', 'p_value']
df_corr.reset_index(drop=True).loc[:,column_order_corr].to_feather(f"{data_dir}/spin-test/replication_spin_test_data.feather")
df_corr_pvalues.reset_index(drop=True).loc[:,column_order_corr_pvalues].to_feather(f"{data_dir}/spin-test/replication_spin_test_pvalues.feather")